# 1. Evaluating LLM performance in Wetlands Tracker

January 29 2024


This notebook contains a workflow to load data extracted by the main branch of this repository - the wetlands-tracker - and perform the following:

- Report overall data quality.
- Compare LLM responses with Regex.
- Print out random LLM samples for human eval.
- Use a fine-tuned OpenAI model to regenerate responses.
- Categorize the responses using `.json` files generated during runtime, or pre-loaded from the repository.
- Export a cleaner version of the `wetland_impact` table.

In [3]:
import pandas as pd
import numpy as np
import os
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
import io
import requests
import json
import tiktoken  # Ensure you have this package installed
from tqdm import tqdm
import openai
import re
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [5]:
OPENAI_API_KEY = ""


### List of errors to track.


1. Step 1: Gets these tables from data schema:

    * raw_df

    * validation_df

    * wetland_df

    * embed_df
    
2. Step 2: errors in **RAW_DF**

    * Number of 'unknowns' by column/table

    * Scraping/URL errors

    * len(longitude list) == len(latitude list) for same noticeID
    
    * No. of special notices


3.  Step 3: errors in **WEtland and Validation df**

    * API based errors (in LLMfunctions.py, get requests error (404, 202 etc.) and put in column)

    * check rows where wetland_impact_unit = 0

    * check rows where regex and OPenAI differ (NOTE - ONLY FOR FUTURE NOTICES)
    
    * Value counts/table of different impact units+types

4. Step 4: errors in embed_df

    * Total project area - check for zero



In [4]:

# Set the working directory
os.chdir(r'D:\Work\Georgetown\acad\mdi\usace\usace_analysis')

# Read parquet
#read
val_df = pd.read_parquet('val_df_5jan.parquet')
wet_df = pd.read_parquet('wet_df_5jan.parquet')
loc_df = pd.read_parquet('loc_df_5jan.parquet')
emb_df = pd.read_parquet('emb_df_5jan.parquet')


## Evaluating LLM responses against Regex

There are two methods:

1. Convert to a symmetrical JSON schema, and compare each value of key.

2. Check the raw string distances of the text captured in the 'values' by the LLM and regex dictionaries.

Use the Levenshtein distance to quantify distance of the strings.

In [ ]:

# # Function to calculate unknown and NaN counts and percentages
# def calculate_unknown_nan(df):
#     results = []

#     for column in df.columns:
#         unknown_count = (df[column] == 'unknown').sum()
#         nan_count = df[column].isna().sum()
#         total_count = len(df)
        
#         unknown_percent = (unknown_count / total_count) * 100
#         nan_percent = (nan_count / total_count) * 100

#         results.append({
#             'Column': column,
#             'Unknown Count': unknown_count,
#             'Unknown Percent': unknown_percent,
#             'NaN Count': nan_count,
#             'NaN Percent': nan_percent
#         })

#     return pd.DataFrame(results)

# # Generate the table
# unknown_nan_table = calculate_unknown_nan(main_df)
# # unknown_nan_table

# main_df['len1'] = main_df.pdf_impact.apply(lambda x: len(x))
# main_df['len2'] = main_df.wetland_llm_dict.apply(lambda x: len(x))

# main_df[['pdf_impact', 'wetland_llm_dict', 'lev_distance', 'len1', 'len2']][(main_df['len1']> 7) & (main_df['len2']> 16)].sort_values(by='lev_distance').tail(1).values

# def determine_message(row):
#     if row['len1'] == 2 and row['len2'] == 16:
#         return 'Both returned NAs'
#     elif row['len1'] == 2 and row['len2'] > 16:
#         return 'Regex returned NAs'
#     elif row['len2'] == 16  and row['len1'] != 2:
#         return 'OpenAI returned NAs'
#     else:
#         return 'Both returned objects'

# # Apply the function to each row
# main_df['message'] = main_df.apply(determine_message, axis=1)
# main_df.message.value_counts()

# df = main_df[['web_title', 'pdf_impact', 'wetland_llm_dict', 'lev_distance', 'len1', 'len2']].copy()


# df.sort_values(by='len1').head(10)

# fig, ax = plt.subplots(1, figsize=(8, 8))

# df[(df.len1<2000)].len1.hist(color='red', alpha=.6, bins=30, legend=True)

# df[(df.len2<2000)].len2.hist(color='blue', alpha=.5, bins=30, legend=True)

## Rewrite prompt + Fine-tuned model

In [9]:
human_prompt = """ You will get a passage about a project. Use the function schema to get a dictionary. 
            For each wetland, provide its type or descriptor, the area it occupies in acres,
            and the duration of its impact (e.g., 'permanent' or 'temporary'). 
            You should focus on sentences which contain information on area or (linear feet) and wetland type impacted.
            The priority is the breakdown by wetland type. 
            Only record 'impact_duration' as 'permanent' or 'temporary' if it appears in the same sentence as 'wetland_type' 
            and 'area', otherwise write 'unknown'. 
            Some passages describe multiple projects, and sequentially describe the wetland impacts for each project.            
            Keywords like 'including' or 'of which', or parentheses '()' may indicate nested projects - Do not double count impacts.
            The impact_type for a wetland is usually loss or damage. 
            Sometimes words like positive, beneficial, restoration etc. will indicate a positive or neutral impact, record it.
            Take a deep breath, and work on this problem step-by-step. 

            Here is the text: """ 

In [10]:
human_prompt2 = """ You will get a passage about a project. Use the function schema to get a dictionary. 
Identify Wetlands and Impacts: Look for sentences detailing the wetland/water/land type and the impacts. Record the type, quantity, and unit. There may be multiple impacts.
Determine Impact Duration: Mark 'permanent' or 'temporary' if mentioned with wetland type and area; else, write 'unknown'.
Assess Impact Type: Record the type of impact being described, preferably recording the exact term.
Avoid Double Counting: Be mindful of nested projects or phrases indicating multiple projects. Do not double count impacts.Some passages describe multiple projects, and/or sequentially describe the wetland impacts for each project.            
Keywords like 'including' or 'of which', or parentheses '()' may indicate nested projects.
Create a Dictionary: For each wetland, provide its type, area, duration, and impact type in a structured format.
Take a deep breath, and work on this problem step-by-step. Here is the text: """ 

In [11]:
llm_prompt = """Your task is to Extract information from a project description to create a structured dictionary, following the provided JSON schema. Focus on identifying wetland impacts based on the given criteria.

JSON Schema Overview:

wetland_type: Type or descriptor of the wetland (e.g., swamp, marsh).
impact_quantity: Numeric value of the impacted area.
impact_unit: Units of measurement (acres, sq. feet, linear feet).
impact_duration: Duration of impact (permanent, temporary, unknown).
impact_type: Nature of impact (harmful, beneficial, unknown).
Instructions:

Identify Wetlands and Impacts: Look for sentences detailing the wetland/water/land type and the impacts. Record the type, quantity, and unit. There may be multiple impacts.
Determine Impact Duration: Mark 'permanent' or 'temporary' if mentioned with wetland type and area; else, write 'unknown'.
Assess Impact Type: Identify if the impact is harmful, beneficial, or unknown.
Avoid Double Counting: Be mindful of nested projects or phrases indicating multiple projects. Do not double count impacts.
Create a Dictionary: For each wetland, provide its type, area, duration, and impact type in a structured format.
Example Input:
'The project will affect 3.5 acres of marshland permanently, leading to habitat loss...'

Expected Output:
{'wetlands': [{'wetland_type': 'marshland', 'impact_quantity': '3.5', 'impact_unit': 'acres', 'impact_duration': 'permanent', 'impact_type': 'loss'}]}
"""

### Chat completion function

In [12]:

@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(GPT_MODEL, messages, API_KEY, functions=None, function_call=None):
    """
    This function makes a POST request to the OpenAI Chat Completion API, sending a JSON payload 
    that includes the GPT model, a series of messages. It is designed to retry up to three times with 
    exponential backoff in case of failures. 

    Parameters:
    GPT_MODEL (str): The identifier of the GPT model to be used for generating responses.
    messages (list): A list of message dictionaries, where each dictionary represents a single exchange 
                     within the chat. Each message has a 'role' (either 'user' or 'assistant') and 
                     'content' (the message text).
    API_KEY (str): The API key for authentication with the OpenAI API.
    functions (list, optional): A list of additional functions to be used along with the GPT model. 
                                Default is None.
    function_call (dict, optional): A dictionary representing a function call, including function name 
                                    and parameters. Default is None.

    Returns:
    requests.models.Response: The response object from the API call if successful.
    Exception: The exception object if the API call fails.
    """
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + API_KEY,
    }
    json_data = {"model": GPT_MODEL, "messages": messages, "temperature": 0}
    if functions is not None:
        json_data.update({"functions": functions})
    if function_call is not None:
        json_data.update({"function_call": function_call})
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e


#############################################################################################

### Function 1 - Wetland impact

main_func = [

            { # function to capture wetland impacts
              "name": "wetland_analysis",
              "description": """Get attributes for project, wetlands, area and duration.""",
              "parameters": {
                "type": "object",
                "properties": {
                   "wetlands": {
                    "type": "array",
                    "description": "Array containing information about project and impact on wetlands etc..",
                    "items": {
                      "type": "object",
                      "properties": {
                        "wetland_type": {
                          "type": "string",
                          "description": "The type or descriptor of the wetland, waters, waterbottoms, swamp, marsh etc."
                        },

                          "impact_quantity": {
                          "type": "string",
                          "description": "The quantity of the wetland, stream or land impacted. Only numeric."
                        },
                          
                        "impact_unit": {
                          "type": "string",
                          "enum": ["acres", "sq. feet", "linear feet"],
                          "description": "The units of the measurement for the impacted area of the wetland. Only text."
                        },

                        "impact_duration": {
                          "type": "string",
                          "enum": ["permanent", "temporary", "unknown"],
                          "description": "Duration of impact or loss of the wetland, e.g., 'permanent' or 'temporary'. Write 'Unknown' if no descriptor in same sentence."
                        },
                        "impact_type": {
                          "type": "string",
                          "description": "Whether project impact is harmful, beneficial or unknown. "
                        }
                          
                      }
                    }
                  }
                }
              }
            }

        ]




def openAIfunc_wetland(input_text,  API_KEY, GPT_MODEL, prompt):
    
    messages = []
   # messages.append({"role": "system", "content": "Work step by step."})   # POTENTIAL ERROR
    messages.append({"role": "user", "content": prompt + input_text                
                    })

# https://arxiv.org/pdf/2309.03409.pdf
    chat_response = chat_completion_request(GPT_MODEL,
        messages, API_KEY, functions=main_func, function_call={"name": "wetland_analysis"} )

    func_response = chat_response.json()["choices"][0]["message"]["function_call"]["arguments"]

    # Convert the stringified JSON to a Python dictionary
#     func_response_dict = json.loads(func_response)

    # except (KeyError, json.JSONDecodeError) as e:
    # print(f"An error occurred: {e}")
    # func_response_dict = {}

    return func_response



## Try on some examples

In [15]:
fine_tuned_model_id = 'ft:gpt-3.5-turbo-0613:eidc-mdi-georgetown:usace-json:8ait0mXa'


In [13]:
text1 = """Initially, the authorized project proposed that  approximately 7.169 acres of  forested swamp wetlands would be temporarily impacted during projec t construction but were reduced to 5.873 acres.  Moreover, the aut horized project also propos ed temporary impacts to approximately 5.755 acres of herbaceous wetl ands but was increased to 6.284 acres during project construction."""


In [ ]:
chat_response = openAIfunc_wetland(input_text=text1, API_KEY=OPENAI_API_KEY, GPT_MODEL = fine_tuned_model_id)
chat_response.json()

### Dec 2023 version

### - Human prompt, GPT-3.5-turbo-0613

In [16]:
chat_response = openAIfunc_wetland(input_text=text1, API_KEY=OPENAI_API_KEY, 
                                   GPT_MODEL = 'gpt-3.5-turbo-0613', prompt=human_prompt)
chat_response

'{\n  "wetlands": [\n    {\n      "wetland_type": "forested swamp",\n      "impact_quantity": "5.873",\n      "impact_unit": "acres",\n      "impact_duration": "temporary",\n      "impact_type": "loss"\n    },\n    {\n      "wetland_type": "herbaceous wetlands",\n      "impact_quantity": "6.284",\n      "impact_unit": "acres",\n      "impact_duration": "temporary",\n      "impact_type": "loss"\n    }\n  ]\n}'

### - LLM prompt, GPT-3.5-turbo-0613-ft-eidc

In [17]:
chat_response = openAIfunc_wetland(input_text=text1, API_KEY=OPENAI_API_KEY, 
                                   GPT_MODEL = fine_tuned_model_id, prompt=human_prompt2)
chat_response

'{\n  "wetlands": [\n    {\n      "wetland_type": "forested swamp wetlands",\n      "impact_quantity": "5.873",\n      "impact_unit": "acres",\n      "impact_duration": "temporary",\n      "impact_type": "loss"\n    },\n    {\n      "wetland_type": "herbaceous wetlands",\n      "impact_quantity": "6.284",\n      "impact_unit": "acres",\n      "impact_duration": "temporary",\n      "impact_type": "loss"\n    }\n  ]\n}'

## Run fine-tuned model on notices from 2020-2024

Run it in three batches for efficiency reasons:

- From 2021-2024 (~1500)

- From 2012 - 2020, batch 1 (~ 2000)

- From 2012 - 2020, batch 2 (~ 2200)


The code below, however, shows how to do it in two batches. Adapt as needed.

In [137]:
wet_df = val_df[['noticeID', 'pdf_character', 'datePublished']].copy()

# Convert 'datePublished' to datetime
wet_df['datePublished'] = pd.to_datetime(wet_df['datePublished'], errors='coerce')
wet_df['year'] = wet_df['datePublished'].dt.year

# Exclude rows with 'unknown', 'ERROR', NaN in 'pdf_character', and length less than 50 characters
wet_df = wet_df[(wet_df.pdf_character != "unknown") & 
                (wet_df.pdf_character.notna()) & 
                (~wet_df.pdf_character.str.contains("ERROR", na=False))]
wet_df['len'] = wet_df.pdf_character.apply(len)
wet_df = wet_df[wet_df.len > 50]

# Partition the DataFrame into two based on the year
wet_df_pre_2020_part1 = wet_df[wet_df.year <= 2020]
wet_df_post_2020_part2 = wet_df[wet_df.year > 2020]


,noticeID,pdf_character,datePublished,year
0,Notice_NO_6585,"Clear, grade, excavate, and place approximatel...",2023-10-30,2023
1,Notice_NO_6578,The applicant proposes to perform dredging ope...,2023-11-09,2023
2,Notice_NO_6594,The applicant has request ed Department of the...,2023-10-16,2023
3,Notice_NO_6589,The applicant is proposing to discharge approx...,2023-10-20,2023
4,Notice_NO_6566,The applicant proposes to expand an existing b...,2023-11-21,2023
...,...,...,...,...
6635,Notice_NO_6649,Theproposed work is to install and maintain a ...,2024-01-01,2024
6636,Notice_NO_6650,The applicant seeks authorization to deconstru...,2023-12-28,2023
6637,Notice_NO_6651,The overall goal of the CAL MB is to re -estab...,2024-01-02,2024
6638,Notice_NO_6652,The overall goal of the,2024-01-02,2024


In [146]:
# Function to process and append results to a DataFrame
def process_and_append_results(df):
    pdf_character_list = df['pdf_character'].tolist()
    wetland_llm_results = []

    for i, pdf_character in tqdm(enumerate(pdf_character_list), total=len(pdf_character_list)):
        try:
            result = openAIfunc_wetland(input_text=pdf_character, 
                                        API_KEY=OPENAI_API_KEY, 
                                        GPT_MODEL=fine_tuned_model_id, 
                                        prompt=human_prompt2)
            wetland_llm_results.append(result)
        except Exception as e:
            print(f'Error processing item {i}: {e}')
            wetland_llm_results.append(None)

    df['wetland_llm_dict'] = wetland_llm_results
    return df

# Process pre-2020 and post-2020 data
wet_df_pre_2020_part1 = process_and_append_results(wet_df_pre_2020_part1)
wet_df_pre_2020_part2 = process_and_append_results(wet_df_pre_2020_part2)


 30%|███████████████████████▌                                                       | 459/1543 [14:17<56:23,  3.12s/it]

Error processing item 459: can only concatenate str (not "NoneType") to str


 35%|███████████████████████████▉                                                   | 546/1543 [17:14<32:41,  1.97s/it]

Error processing item 546: can only concatenate str (not "NoneType") to str


 98%|████████████████████████████████████████████████████████████████████████████▍ | 1513/1543 [56:41<01:00,  2.01s/it]

Error processing item 1513: can only concatenate str (not "NoneType") to str


 99%|█████████████████████████████████████████████████████████████████████████████▎| 1530/1543 [56:59<00:14,  1.10s/it]

Error processing item 1530: can only concatenate str (not "NoneType") to str


100%|██████████████████████████████████████████████████████████████████████████████| 1543/1543 [57:36<00:00,  2.24s/it]


### JSON to columns

In [64]:
import pandas as pd
import json

def dict_to_columns(df_source, dict_col, index_cols):
    # Initialize an empty DataFrame for the expanded data
    df_main = pd.DataFrame()

    # Loop through each row in the source DataFrame
    for _, row in df_source.iterrows():
        # Extract the item from the dict_col
        dict_item = row[dict_col]

        # Skip rows where dict_item is None
        if dict_item is None:
            continue

        # Check if the item in dict_col is a string and convert it to a dictionary
        if isinstance(dict_item, str):
            try:
                dict_item = json.loads(dict_item)
            except json.JSONDecodeError:
                print(f"Error decoding JSON for row: {row}")
                continue

        # Use the dictionary to extract data
        wetland_dict = dict_item.get('wetlands', [])
        ids = {col: row[col] for col in index_cols}

        # Create a DataFrame for the wetlands data
        wetlands_df = pd.DataFrame(wetland_dict)

        # Handle the case where no wetlands data is present
#         if wetlands_df.empty:
#             wetlands_df = pd.DataFrame(columns=['wetland_type', 'area', 'impact_duration', 'impact_type'])
        
        # Add ID columns
        for col, id_val in ids.items():
            wetlands_df[col] = id_val

        # Append to the main DataFrame
        df_main = pd.concat([df_main, wetlands_df], ignore_index=True)
        
    return df_main

# B. Break out dictionary to columns
wetland_impact_df1 = dict_to_columns(df_source=wet_df_pre_2020_part1, dict_col='wetland_llm_dict', index_cols=['noticeID'])
wetland_impact_df2 = dict_to_columns(df_source=wet_df_pre_2020_part2, dict_col='wetland_llm_dict', index_cols=['noticeID'])


In [ ]:

wetland_impact_df1.to_parquet('wetland_impacts_pre_2020_1.parquet')

wetland_impact_df2.to_parquet('wetland_impacts_pre_2020_2.parquet')

In [97]:
wet_df_12_20_part1[[ 'noticeID', 'pdf_character', 'wetland_llm_dict']].sample(1).values

array([['Notice_NO_4798',
        'Construct a 2 5-feet by 30- feet pile-supported boathousewitha 4-feet by 30- feet wharf , and replace approximately 45 linear feet of existing bulkhead, all for private recreational use.The proposed boathouse and wharf is to be located approximately perpendicular to the mean high water shoreline, the outer most edge toextend approximately 30 feet channelward therefrom.Project implementation would impact approximately 0.02 acres of water bottoms .Written comments, including suggestions for modifications or objections to the proposed work, stating reasons therefore , are being solicited from anyone having interest in this permit request .',
        '{\n  "wetlands": [\n    {\n      "wetland_type": "water bottoms",\n      "impact_quantity": "0.02",\n      "impact_unit": "acres",\n      "impact_duration": "unknown",\n      "impact_type": "loss"\n    }\n  ]\n}']],
      dtype=object)

## 1. Cleaning Impact types

How to categorize the impact types?

Which impact type *verbs* can be defined as losses of wetland?
We can either hardcode it manually, or rely on the LLM.

Even for hardcoding, we need to reduce the number of similar strings to a few standard phrases.
- How? Semming, lemmatization, embedding/clustering. (compare to the knowledge base)



In [30]:
wet_impacts_21_24 = pd.read_parquet('wetland_impact_df_20.parquet')

wet_impacts_12_17 = pd.read_parquet('wetland_impacts_pre_2020_1.parquet')

wet_impacts_17_20 = pd.read_parquet('wetland_impacts_pre_2020_2.parquet')

wet_impacts = pd.concat([wet_impacts_21_24, wet_impacts_17_20, wet_impacts_12_17], axis=0)

In [71]:
## import re
# wet_impacts.impact_type.value_counts()
# Example list of unique impact types
unique_impact_types = list(wet_impacts.impact_type.unique())

# Initialize lists for categories
loss_list = []
benefit_list = []
uncategorized_list = []

# Define keywords for each category with regular expression patterns
loss_keywords = [r'\bloss\b', r'\bfill', r'\bdredg','impact', r'\bdirect\b', 
                 r'\bexcavat', r'\bremov', r'\bclear', r'\bdischarg']
benefit_keywords = [r'\brestor', r'\bcreat', r'\benhanc', r'\bpreserv']

# Categorize each impact type
for impact in unique_impact_types:
    if any(re.search(keyword, impact) for keyword in loss_keywords):
        loss_list.append(impact)
    elif any(re.search(keyword, impact) for keyword in benefit_keywords):
        benefit_list.append(impact)
    else:
        uncategorized_list.append(impact)

# # Now you have three lists: loss_list, benefit_list, uncategorized_list
# wet_impacts[wet_impacts.impact_type.isin(uncategorized_list)].impact_type.value_counts().head(40)

# Assuming loss_list, benefit_list, uncategorized_list are already populated as per your previous code

impact_categs = {
    "LOSS": loss_list,
    "BENEFIT": benefit_list,
    "UNCATEGORIZED": uncategorized_list
}

# Splitting the data
loss_data = {"LOSS": impact_categs["LOSS"]}
benefit_data = {"BENEFIT": impact_categs["BENEFIT"]}
uncategorized_data = {"UNCATEGORIZED": impact_categs["UNCATEGORIZED"]}

# Dictionary containing each category and its data
data_categories = {
    "loss": loss_data,
    "benefit": benefit_data,
    "uncategorized": uncategorized_data
}

# Base name for file naming
base_name = "impact_type"

# Loop through each category and save to a separate JSON file
for category, data in data_categories.items():
    file_name = f"{base_name}_{category}.json"
    with open(file_name, 'w') as file:
        json.dump(data, file, indent=4)
        
        
# Function to categorize impact based on pre-categorized lists
def categorize_impact(row):
    impact = row['impact_type']
    if impact in loss_list:
        return "LOSS"
    elif impact in benefit_list:
        return "BENEFIT"
    else:
        return "UNCATEGORIZED"

# Applying the function to the DataFrame
wet_impacts['impact_type_clean'] = wet_impacts.apply(categorize_impact, axis=1)


# # Initialize Porter Stemmer
# stemmer = PorterStemmer()

# # Initialize lemmatizer
# lemmatizer = WordNetLemmatizer()

# def preprocess_text(text):
#     # Convert to lowercase and remove punctuation
#     text = text.lower().translate(str.maketrans('', '', string.punctuation))
    
#     # Tokenize
#     tokens = word_tokenize(text)
    
#     # Lemmatize each token
# #     lemmatized = [lemmatizer.lemmatize(token) for token in tokens]
  
    
#     # Stem each token
#     stemmed = [stemmer.stem(token) for token in tokens]
    
#     # Reconstruct the processed string
#     return ' '.join(stemmed)

# # Apply preprocessing to each item in the 'impact_type' column and create a new column
# wet_impacts['st_impact_type'] = wet_impacts['impact_type'].apply(preprocess_text)


# # wet_impacts[wet_impacts.st_impact_type.str.contains('fill')].impact_type.unique()


### Read in `impact_type.json` (HYPERLINK) to categorize.

## 2. Cleaning Wetland type

In [76]:

# Define keywords for wetland categorization
wetland_keywords = [r'wetland', r'\bjurisdiction', r'\b404\b', r'\bwaters?\b', 
                    r'\bpalustrine\b', r'\bpfo\b', r'\bpem\b', r'\bpss\b', r'\bemergent\b', 
                    r'\bforested\b', r'\bwater\s?bottoms?\b', r'\bestuarine\b', 
                    r'\bherbaceous\b', r'\bmarsh', r'\bstream\s?\b', r'\bhardwood\s?\b']

# Initialize lists for categories
wetland_list = []
uncategorized_list = []

# Extract unique wetland types
unique_wetland_types = list(wet_impacts.wetland_type.unique())
unique_wetland_types = [x.lower() for x in unique_wetland_types]
# Categorize each wetland type
for wetland_type in unique_wetland_types:
    if any(re.search(keyword, wetland_type) for keyword in wetland_keywords):
        wetland_list.append(wetland_type)
    else:
        uncategorized_list.append(wetland_type)

# Create a dictionary with the two categories
wetland_categs = {
    "WETLAND": wetland_list,
    "UNCATEGORIZED": uncategorized_list
}

# Save this dictionary as a JSON file
with open('wetland_categ.json', 'w') as file:
    json.dump(wetland_categs, file, indent=4)
    

### Read in `wetland_categ.json` to categorize

In [78]:
# Load the wetland categories from the JSON file
with open('wetland_categ.json', 'r') as file:
    wetland_categs = json.load(file)

    
# Function to determine wetland_clean
def determine_wetland_clean(row):
    wetland_type = row['wetland_type'].lower() 
    # Check if wetland_type is in the 'WETLAND' category and not 'unknown' or 'uncategorized'
    if wetland_type in wetland_categs["WETLAND"]:
        return 'wetland'
    else:
        return 'uncategorized'

# Apply the function to the DataFrame
wet_impacts['wetland_clean'] = wet_impacts.apply(determine_wetland_clean, axis=1)
wet_impacts

,wetland_type,impact_quantity,impact_unit,impact_duration,impact_type,noticeID,impact_type_clean,wetland_clean
0,jurisdictional wetlands,9.85,acres,permanent,loss,Notice_NO_6585,LOSS,wetland
1,dock slip,"74,643",cubic yards,unknown,dredging,Notice_NO_6578,LOSS,uncategorized
2,Section 10 waters,unknown,unknown,unknown,structures,Notice_NO_6594,UNCATEGORIZED,wetland
3,wetlands,2.79,acres,unknown,fill,Notice_NO_6589,LOSS,wetland
4,wetlands,0.61,acres,unknown,fill,Notice_NO_6589,LOSS,wetland
...,...,...,...,...,...,...,...,...
3867,forested wetlands,0.26,acres,unknown,loss,Notice_NO_2484,LOSS,wetland
3868,waters,Mobile Bay,unknown,unknown,loss,Notice_NO_4848,LOSS,wetland
3869,wetlands,12.43,acres,unknown,fill,Notice_NO_4849,LOSS,wetland
3870,jurisdictional waters,0.66,acres,unknown,fill,Notice_NO_467,LOSS,wetland


In [79]:
wet_impacts.wetland_clean.value_counts()

wetland_clean
wetland          6905
uncategorized    6024
Name: count, dtype: int64

In [80]:

# Categorize impacts according to both wetland and impact_type
def categorize_impact(row):
    if row['impact_type_clean'] == 'LOSS' and row['wetland_clean'] == 'wetland':
        return 'LOSS'
    elif row['impact_type_clean'] == 'BENEFIT' and row['wetland_clean'] == 'wetland':
        return 'BENEFIT'
    return 'UNCATEGORIZED'  # Default category if conditions are not met

# Apply the modified function to the DataFrame
wet_impacts['impact_type_clean'] = wet_impacts.apply(categorize_impact, axis=1)
wet_impacts

,wetland_type,impact_quantity,impact_unit,impact_duration,impact_type,noticeID,impact_type_clean,wetland_clean
0,jurisdictional wetlands,9.85,acres,permanent,loss,Notice_NO_6585,LOSS,wetland
1,dock slip,"74,643",cubic yards,unknown,dredging,Notice_NO_6578,UNCATEGORIZED,uncategorized
2,Section 10 waters,unknown,unknown,unknown,structures,Notice_NO_6594,UNCATEGORIZED,wetland
3,wetlands,2.79,acres,unknown,fill,Notice_NO_6589,LOSS,wetland
4,wetlands,0.61,acres,unknown,fill,Notice_NO_6589,LOSS,wetland
...,...,...,...,...,...,...,...,...
3867,forested wetlands,0.26,acres,unknown,loss,Notice_NO_2484,LOSS,wetland
3868,waters,Mobile Bay,unknown,unknown,loss,Notice_NO_4848,LOSS,wetland
3869,wetlands,12.43,acres,unknown,fill,Notice_NO_4849,LOSS,wetland
3870,jurisdictional waters,0.66,acres,unknown,fill,Notice_NO_467,LOSS,wetland


In [81]:
wet_impacts.impact_type_clean.value_counts()

impact_type_clean
UNCATEGORIZED    7253
LOSS             5384
BENEFIT           292
Name: count, dtype: int64

## 3. Cleaning Impact units


* Categorize units into a few standardized ones: acre, sq. feet, miles, cubic years, feet.

In [82]:

def standardize_unit(unit):
    patterns = {
        'acres': re.compile(r'\bacres?\b', re.IGNORECASE),
        'square feet': re.compile(r'\bsq\.?\s*ft\.?|\bft2\b|\bft²\b|\bsquare\s*feet\b|\bfeet²\b|\bsq\.?\s*feet\.?\b', re.IGNORECASE),
        'feet': re.compile(r'\bfeet\b|\bfoot\b|\bft\b', re.IGNORECASE),
        'cubic yards': re.compile(r'\bcubic\s*yards\b|\byd3\b|\byards³\b|\byards3\b|\bcy\b|\bcubic-yards\b', re.IGNORECASE),
        'miles': re.compile(r'\bmiles?\b', re.IGNORECASE)
    }

    for standardized_unit, pattern in patterns.items():
        if pattern.search(unit):
            return standardized_unit

    return unit

wet_impacts['impact_unit_clean'] = wet_impacts['impact_unit'].apply(lambda x: standardize_unit(x.lower().strip()))

# List of specified units
clean_units = ['acres', 'linear feet', 'square feet', 'unknown', 'cubic yards', 'feet', 'miles']

# Replace values not in specified categories with 'other'
wet_impacts['impact_unit_clean'] = wet_impacts['impact_unit_clean'].apply(lambda x: x if x in 
                                                                                        clean_units else 'other')


wet_impacts.impact_unit_clean.value_counts().head(15)

impact_unit_clean
acres          7618
feet           1584
square feet    1081
unknown         903
other           821
cubic yards     816
miles           106
Name: count, dtype: int64

In [ ]:
# compare with old

# wet_impacts_old = pd.read_csv(r'D:\Work\Georgetown\acad\mdi\usace\usace_analysis\data_download\wetland_final_df.csv')

# wet_impacts_old.impact_unit.value_counts()

# wet_impacts.impact_unit.value_counts()

# wet_impacts_old.wetland_type.value_counts()

# wet_impacts.wetland_type.value_counts()
# wet_impacts_old.impact_type.value_counts()

# wet_impacts.impact_type.value_counts().head(30)

### Wetland impact errors

Explicitly test for the following types of errors. These 'patterns' were spotted by looking at some random results.

- Double counting

- Multi-unit

In [83]:

# Function to convert each group into a JSON string
def group_to_json(group):
    group_dict = group.drop('noticeID', axis=1).to_dict(orient='records')
    return json.dumps({'wetlands': group_dict})

# Group by noticeID and apply the function
wdf = wet_impacts.groupby('noticeID').apply(group_to_json).reset_index(name='json_data')

# Assuming df is your DataFrame with a column 'json_data' containing JSON strings

# Function to parse JSON and return the number of objects
def count_impacts(json_str):
    try:
        wetlands = json.loads(json_str)['wetlands']
        return len(wetlands)
    except:
        return 0

# Add the impact_count column
wdf['impact_count'] = wdf['json_data'].apply(count_impacts)

# Function to analyze impacts and set doub_count
def analyze_and_set_doub_count(row):
    try:
        wetlands = json.loads(row['json_data'])['wetlands']
        quantities = [w['impact_quantity'] for w in wetlands if 'impact_quantity' in w and w['impact_quantity'] is not None]
        max_impact = max(quantities, default=0)
        sum_of_others = sum(q for q in quantities if q != max_impact)

        # Allowing a 1-2% margin of error
        if max_impact != 0 and abs(max_impact - sum_of_others) / max_impact <= 0.01:
            return 1
        else:
            return 0
    except:
        return 0

# Add the doub_count column
wdf['doub_count'] = wdf.apply(analyze_and_set_doub_count, axis=1)

error_notices1 = list(wdf[wdf['doub_count']==1].noticeID.unique())

print('Number of notices with likely double counts:', len(error_notices1))

Number of notices with likely double counts: 0


In [84]:
wdf = wdf.merge(val_df[['noticeID', 'pdf_character']], on='noticeID', how='inner')

# Assuming df is your DataFrame and it has a column named 'pdf_character'

# Regular expressions for different units
units_regex = {
    re.compile(r'\bacr(es|e)?\b', re.IGNORECASE): 'acre',
    re.compile(r'\bsquare feet\b|\bsq\.? feet\b|\bsq\.? ft\b|\bsquare foot\b', re.IGNORECASE): 'square feet'}


def check_multi_units(text):
    try:
        found_categories = set()
        for regex, category in units_regex.items():
            if regex.search(text):
                found_categories.add(category)
    except: return 0
    return 1 if len(found_categories) > 1 else 0


# Apply the function to create the multi_unit column
wdf['multi_unit'] = wdf['pdf_character'].apply(check_multi_units)

# get unique noticeIDs
error_notices2 = list(wdf[wdf['multi_unit']==1].noticeID.unique())
#append both lists
error_notices = list(set(error_notices1+ error_notices2))
print('Number of notices with Multi unit errors:', len(error_notices2))

Number of notices with Multi unit errors: 246


## 4. Cleaning Impact quantity


Replace any commas, get rid of 0 impacts, convert to float.

In [87]:

# Extracting the number, handling commas, decimals, NaNs, and blanks
wet_impacts['impact_qty_num'] = wet_impacts['impact_quantity'].str.extract(r'([\d,.]+)').replace(',', '', regex=True)
wet_impacts['impact_qty_num'] = pd.to_numeric(wet_impacts['impact_qty_num'], errors='coerce', downcast='float')

#replace 0.00 values
wet_impacts['impact_qty_num'] = wet_impacts['impact_qty_num'].replace(0.00, np.nan)

# wet_impacts[wet_impacts.impact_quantity.str.contains(r'([\d,.]+)', regex=True)][['impact_qty_num', 'impact_quantity']]

# # export
# wetland_final_df = wet_impacts[['noticeID', 'wetland_type', 'impact_duration', 'impact_qty_num',
#                                 'impact_unit_clean', 'impact_type', 'impact_type_clean']]
# wetland_final_df = wetland_final_df.rename(columns={'impact_qty_num': 'impact_quantity', 'impact_unit_clean': 'impact_unit'})

# # wetland_final_df.to_csv('wetland_final_df_ft_16Jan.csv')


In [89]:
wet_impacts['error_likely'] = wet_impacts['noticeID'].isin(error_notices).astype(int)


## Final export

`wetland_impact_clean.parquet` will be loaded and merged to the main dataset into the next notebook.

In [91]:
wet_impacts.to_parquet('wetland_impact_clean.parquet')

In [90]:
wet_impacts

,wetland_type,impact_quantity,impact_unit,impact_duration,impact_type,noticeID,impact_type_clean,wetland_clean,impact_unit_clean,impact_qty_num,error_likely
0,jurisdictional wetlands,9.85,acres,permanent,loss,Notice_NO_6585,LOSS,wetland,acres,9.85,0
1,dock slip,"74,643",cubic yards,unknown,dredging,Notice_NO_6578,UNCATEGORIZED,uncategorized,cubic yards,74643.00,0
2,Section 10 waters,unknown,unknown,unknown,structures,Notice_NO_6594,UNCATEGORIZED,wetland,unknown,NaN,0
3,wetlands,2.79,acres,unknown,fill,Notice_NO_6589,LOSS,wetland,acres,2.79,0
4,wetlands,0.61,acres,unknown,fill,Notice_NO_6589,LOSS,wetland,acres,0.61,0
...,...,...,...,...,...,...,...,...,...,...,...
3867,forested wetlands,0.26,acres,unknown,loss,Notice_NO_2484,LOSS,wetland,acres,0.26,0
3868,waters,Mobile Bay,unknown,unknown,loss,Notice_NO_4848,LOSS,wetland,unknown,NaN,0
3869,wetlands,12.43,acres,unknown,fill,Notice_NO_4849,LOSS,wetland,acres,12.43,0
3870,jurisdictional waters,0.66,acres,unknown,fill,Notice_NO_467,LOSS,wetland,acres,0.66,0




## END


## Project categorization

In [5]:
def openai_embed(input_text, API_KEY):
    """
    This function initializes an OpenAI client with the provided API key and then generates embeddings 
    for the specified input text using the 'text-embedding-ada-002' model. It counts the number of tokens 
    in the input text, generates embeddings, and extracts the embedding vector from the response.

    Parameters:
    input_text (str): The text for which embeddings are to be generated.
    API_KEY (str): The API key for authentication with the OpenAI API.

    Returns:
    A tuple containing two elements:
           1. token_count (int): The number of tokens in the input text.
           2. vector (list or str): The embedding vector if successful, or an error message if an exception occurs.

    Raises:
    Exception: Captures and returns any exceptions that occur during the API call or processing of the response.
    """

    # initialize client
    client = OpenAI(
    api_key=API_KEY,  
    )
    try:
        openai.api_key= API_KEY
        encoding = tiktoken.encoding_for_model('text-embedding-ada-002')
        #count number of tokens
        token_count =  len(encoding.encode(input_text))

        #generate embeddings  
        response = client.embeddings.create(input=input_text, model='text-embedding-ada-002')
        
        embeddings = response.model_dump_json(indent=0)
        
    #     embeddings = embeddings.values[0]
        
        # Load the JSON data
        data = json.loads(embeddings)

        # Access and extract the vector
        vector = data["data"][0]["embedding"]
        return token_count, vector
    
    except Exception as e:
            # Return the error message in place of embeddings
        return token_count, f"Error: {str(e)}"



In [6]:
descr = ["""Commercial developments - Include non-manufacturing business establishments 
such as department stores, hardware stores, retail outlets, grocery stores, car washes, corner stores, 
office buildings, strip malls, shopping centers, movie theaters, hotels/motels/inns, hospitals, etc""",

"""Drainage features - Drainage features  include gravity drainage channels and canals; water control structures; and pump stations and associated structures.""",

"""Industrial developments - Industrial developments are defined as facilities used to produce goods in connection with, 
or as part of, a process or system.  Such developments incl ude refineries, steel mills, 
shipyards, fabrication facilities, food processing facilities, bulk loading facilities,
landfills, water treatment systems, etc.""",

"""Oil and Gas facilities - Oil and Gas facilities located within the coastal zone include 
(for the purposes of this guide) well sites, production facilities and storage facilities.  """,

"""Pipelines and flowlines - Pipelines and flowlines (hereafter referred to as “lines”) are linear features installed for 
the purpose of transporting materials from one location to another.  Lines can be of any diameter 
and length and any type of liquid or gaseous material can be transported within them.""", 

"""Recreation facilities  - Recreation facilities include, but are not limited to, parks; visitor centers; picnic areas; ball 
fields; playgrounds; public golf courses; community swimming pools, tennis courts and basketball courts; 
and nature, hiking and bike trails.""",

"""Residential subdivisions - residential subdivisions  as multi -house/unit residential developments.""" ,

"""Transportation - Transportation features include roads, bridges and ferries, construction and maintenance of 
which typically are undertaken by state or local governmental bodies, or in the case of ferries, 
private companies.  This guide is focused more transportation features constructe d by municipal entities. 
For the purposes of application processing, air and rail developments should refer to our  
commercial or industrial developments guides depending on the nature of the activity; boat
traffic should refer to our M arinas, Ports or Recreational Facilities guides depending on 
the nature of the activity; and bike and foot trails should refer to  the R ecreational Facilities  guide.""",

"""Utility - Utility activities include potable water facilities and lines, sewerage facilities and lines, gas and electricity facilities 
and lines, phone lines, cable lines and fiber optic lines.""",

"""Ports  - a port as an industrial type, water -based cargo transfer facility """,

"""Levee - A levee is defined as an embankment or wall to control or prevent water movement, to retain water 
or other material, or to raise a road or other lineal use above normal or flood water levels.  
Examples include levees, dikes, flood walls and embankments of any kind.""",

"""Marina - marinas  as any type of development focused on providing water access and docking services to the boating 
community.   Marina amenities  include fueling stations, pump -out stations, wash stations, ice houses, seafood processing 
facilities (including fish cleaning stations), stores, bait shops, restaurants, lodging, etc.  Shipyards and other exclusive
retail/service type facilities such as boat retail and/or repair are not considered marinas for the purposes of this guide."""]


In [10]:
import pandas as pd
import json

# Step 1: Prepare the DataFrame
categories = [desc.split(" - ")[0] for desc in descr]
descriptions = [desc for desc in descr]
df = pd.DataFrame({'desc': descriptions, 'cat': categories})
df

,desc,cat
0,Commercial developments - Include non-manufact...,Commercial developments
1,Drainage features - Drainage features include...,Drainage features
2,Industrial developments - Industrial developme...,Industrial developments
3,Oil and Gas facilities - Oil and Gas facilitie...,Oil and Gas facilities
4,Pipelines and flowlines - Pipelines and flowli...,Pipelines and flowlines
5,Recreation facilities - Recreation facilities...,Recreation facilities
6,Residential subdivisions - residential subdivi...,Residential subdivisions
7,Transportation - Transportation features inclu...,Transportation
8,Utility - Utility activities include potable w...,Utility
9,"Ports - a port as an industrial type, water -...",Ports


In [24]:

# Define your API key
API_KEY = OPENAI_API_KEY 
#= "sk-"


# Step 3: Add Embeddings to the DataFrame
df['cat_embed'] = df['desc'].apply(lambda x: openai_embed(x, API_KEY)[1])


#### Pay attention to data structures. One is a list inside a string '[a,b,c]', or technically, just one string. So is the JSON

In [52]:
emb_df['char_embed'] = emb_df.embeddings.apply(ast.literal_eval)



In [56]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

# Normalize embeddings for cosine similarity calculation
def normalize_embeddings(embeddings):
 # Apply ast.literal_eval to each string in the array to convert it to a list of floats
#     embeddings = [ast.literal_eval(emb) if isinstance(emb, str) else emb for emb in embeddings]
    norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
    return embeddings / norms

# Extract and normalize embeddings
df_embeddings = np.vstack(df['cat_embed'].apply(np.array))
emb_df_embeddings = np.vstack(emb_df['char_embed'].apply(np.array))

normalized_df_embeddings = normalize_embeddings(df_embeddings)
normalized_emb_df_embeddings = normalize_embeddings(emb_df_embeddings)

# Calculate cosine similarity matrix
similarity_matrix = cosine_similarity(normalized_emb_df_embeddings, normalized_df_embeddings)

# Initialize lists to store results
cat_search = []
scores = []

# Iterate over each item in emb_df using tqdm for progress tracking
for idx in tqdm(range(similarity_matrix.shape[0]), desc="Calculating similarities"):
    most_similar_idx = np.argmax(similarity_matrix[idx])
    most_similar_score = similarity_matrix[idx, most_similar_idx]

    # Append the corresponding category and score
    cat_search.append(df.iloc[most_similar_idx]['cat'])
    scores.append(most_similar_score)

# Assign the results to emb_df
emb_df['cat_search'] = cat_search
emb_df['score'] = scores



Calculating similarities: 100%|█████████████████████████████████████████████████| 6506/6506 [00:00<00:00, 19421.09it/s]
